# Bygg din egen Sten-Sax-Påse AI 🤖✊✋✌️

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DITT-REPO/07_rps_kamera.ipynb)

Du har lärt dig hur CNN:er fungerar. Nu bygger du ett från grunden — tränat på **dina egna bilder**.

## Vad vi gör
1. Kopplar Google Drive och hämtar träningsdata
2. Tränar ett CNN på bilder av sten, sax och påse
3. Testar modellen live med din webcam

## Vad du behöver
- Ett Google-konto med Drive
- En kamera (inbyggd eller extern)
- Kör cellerna uppifrån och ned, en i taget

In [ ]:
import os
import json
import base64
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

from IPython.display import display, HTML
from google.colab import output as colab_output

print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")
print("Allt importerat! ✓")

---
## Del 1: Förbehandling

Samma funktion används för **träningsbilder** och **live-kamerabilder** — annars tränar vi på en typ av data men testar på en annan.

In [ ]:
IMG_SIZE = 150  # Alla bilder skalas till 150×150 pixlar
KLASSER  = ['rock', 'paper', 'scissors']

def forbehandla_bild(img):
    """
    Tar en numpy RGB-array (valfri storlek) och returnerar
    en normaliserad float32-array med formen (150, 150, 3).
    Används vid både träning och live-inferens.
    """
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img.astype('float32') / 255.0

# Snabbtest
_dummy = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
assert forbehandla_bild(_dummy).shape == (IMG_SIZE, IMG_SIZE, 3)
assert forbehandla_bild(_dummy).max() <= 1.0
print(f"forbehandla_bild(): {IMG_SIZE}×{IMG_SIZE} px, normaliserad till 0–1 ✓")

---
## Del 2: Google Drive

Kör cellen nedan. En popup öppnas — logga in med ditt Google-konto och ge tillstånd.

Dina bilder sparas i Drive så att de finns kvar nästa gång du öppnar Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_ROOT   = '/content/drive/My Drive'
EXEMPEL_DIR  = f'{DRIVE_ROOT}/rps_exempel'
EGET_DIR     = f'{DRIVE_ROOT}/rps_eget'

print(f"Drive kopplat! ✓")
print(f"Exempeldata: {EXEMPEL_DIR}")
print(f"Egna bilder: {EGET_DIR}")

---
## Del 3: Exempeldata

Kör den här cellen **en gång** för att fylla dina Drive-mappar med exempelbilder från ett färdigt dataset.

Mappstrukturen som skapas:
```
My Drive/
  rps_exempel/
    rock/        ← ~150 bilder
    paper/       ← ~150 bilder
    scissors/    ← ~150 bilder
  rps_eget/
    rock/        ← tom (fyll på med dina egna bilder!)
    paper/       ← tom
    scissors/    ← tom
```

In [ ]:
def skapa_mappar(base_dir):
    for klass in KLASSER:
        os.makedirs(f'{base_dir}/{klass}', exist_ok=True)

skapa_mappar(EXEMPEL_DIR)
skapa_mappar(EGET_DIR)

def spara_exempelbilder(data_dir, bilder_per_klass=150):
    """Hämtar rock_paper_scissors från TF Datasets och sparar som JPG till Drive."""
    ds = tfds.load('rock_paper_scissors', split='train', as_supervised=True)

    räknare = [0, 0, 0]
    mål     = bilder_per_klass

    for img_tensor, label_tensor in ds:
        i = int(label_tensor.numpy())
        if räknare[i] >= mål:
            continue

        img_pil = Image.fromarray(img_tensor.numpy().astype(np.uint8))
        sökväg  = f'{data_dir}/{KLASSER[i]}/{räknare[i]:04d}.jpg'
        img_pil.save(sökväg, quality=90)
        räknare[i] += 1

        if all(r >= mål for r in räknare):
            break

    for i, klass in enumerate(KLASSER):
        print(f"  {klass}: {räknare[i]} bilder sparade")

# Guard: hoppa över om mappen redan är ifylld
rock_dir    = Path(f'{EXEMPEL_DIR}/rock')
befintliga  = len(list(rock_dir.glob('*.jpg')))

if befintliga >= 50:
    print(f"Exempelmappar redan ifyllda ({befintliga} bilder i rock/). Hoppar över. ✓")
else:
    print("Laddar ner exempelbilder från TF Datasets...")
    print("(Tar ~1–2 minuter — körs bara en gång.)\n")
    spara_exempelbilder(EXEMPEL_DIR)
    print(f"\nKlart! Bilderna ligger nu i Drive under rps_exempel/")

---
## Del 4: Välj dataset

Ändra `DATASET` nedan för att välja vilka bilder modellen ska tränas på.

| Värde | Bilder |
|-------|--------|
| `'exempel'` | De 150 exempelbilder vi just laddade ner |
| `'eget'` | Dina egna bilder i `rps_eget/` |

**Hur lägger man till egna bilder?**
1. Öppna [drive.google.com](https://drive.google.com)
2. Navigera till `rps_eget/rock/` och ladda upp foton av en knytnäve
3. Gör samma för `paper/` och `scissors/`
4. Minst 30 bilder per klass rekommenderas — ju fler desto bättre
5. JPG, PNG, WebP — alla format fungerar

In [ ]:
DATASET = 'exempel'  # ← Ändra till 'eget' när du har egna bilder

if DATASET == 'exempel':
    DATA_DIR = EXEMPEL_DIR
elif DATASET == 'eget':
    DATA_DIR = EGET_DIR
else:
    raise ValueError(f"DATASET måste vara 'exempel' eller 'eget', fick: '{DATASET}'")

print(f"Använder dataset: '{DATASET}'")
print(f"Sökväg: {DATA_DIR}\n")

for klass in KLASSER:
    klass_path = Path(f'{DATA_DIR}/{klass}')
    filer = (list(klass_path.glob('*.jpg'))  +
             list(klass_path.glob('*.jpeg')) +
             list(klass_path.glob('*.png'))  +
             list(klass_path.glob('*.webp')))
    print(f"  {klass}: {len(filer)} bilder")

---
## Del 5: Ladda och träna

In [ ]:
def ladda_bilder(data_dir):
    """
    Laddar alla bilder från data_dir/{rock,paper,scissors}/.
    Returnerar X (numpy-array) och y (klassetiketter).
    Stödjer JPG, JPEG, PNG, WebP.
    """
    X, y = [], []
    fel  = 0

    for i, klass in enumerate(KLASSER):
        klass_dir = Path(f'{data_dir}/{klass}')
        filer = (list(klass_dir.glob('*.jpg'))  +
                 list(klass_dir.glob('*.jpeg')) +
                 list(klass_dir.glob('*.png'))  +
                 list(klass_dir.glob('*.webp')))

        laddade = 0
        for fil in filer:
            try:
                img = Image.open(fil).convert('RGB')  # hanterar RGBA, gråskala etc.
                img = forbehandla_bild(np.array(img))
                X.append(img)
                y.append(i)
                laddade += 1
            except Exception as e:
                print(f"  Varning: kunde inte ladda {fil.name}: {e}")
                fel += 1

        print(f"  {klass}: {laddade} bilder laddade")

    if fel:
        print(f"  ({fel} filer hoppades över)")

    return np.array(X, dtype='float32'), np.array(y, dtype='int32')

print("Laddar bilder från Drive...")
X, y = ladda_bilder(DATA_DIR)
print(f"\nX.shape: {X.shape}   y.shape: {y.shape}")

In [ ]:
# Visa 12 slumpmässiga bilder
fig, axes = plt.subplots(2, 6, figsize=(15, 5))
idx = np.random.choice(len(X), 12, replace=False)

for ax, i in zip(axes.flatten(), idx):
    ax.imshow(X[i])
    ax.set_title(KLASSER[y[i]])
    ax.axis('off')

plt.suptitle('Exempelbilder från dataset', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Träning:    {len(X_train)} bilder")
print(f"Validering: {len(X_val)} bilder")

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # rock, paper, scissors
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
print("Tränar modellen...")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32,
    verbose=1
)

_, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"\nValiderings-accuracy: {val_acc*100:.1f}%")

---
## Del 6: Utvärdera modellen

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history.history['loss'],     label='Träning')
axes[0].plot(history.history['val_loss'], label='Validering')
axes[0].set_title('Förlust (Loss)')
axes[0].set_xlabel('Epoch')
axes[0].legend()

axes[1].plot(history.history['accuracy'],     label='Träning')
axes[1].plot(history.history['val_accuracy'], label='Validering')
axes[1].set_title('Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].legend()

plt.tight_layout()
plt.show()

# Confusion matrix
y_pred = np.argmax(model.predict(X_val, verbose=0), axis=1)
cm     = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=KLASSER, yticklabels=KLASSER)
plt.title('Confusion Matrix (validering)')
plt.ylabel('Sant')
plt.xlabel('Förutsagt')
plt.tight_layout()
plt.show()

---
## Del 7: Testa med webcam i realtid! 📷

Kör cellen nedan för att definiera kamera-funktionen, och sedan den sista cellen för att starta den.

Håll upp handen med sten ✊, påse ✋ eller sax ✌️ framför kameran!

In [ ]:
def starta_kamera(model, klassnamn=None):
    """
    Startar en live-kamerafeed i Colab och klassificerar varje frame.
    Visar procentuella sannolikheter uppdaterade var 500ms.

    Args:
        model:     tränad Keras-modell med softmax-output
        klassnamn: lista med klassnamn, t.ex. ['rock', 'paper', 'scissors']
    """
    if klassnamn is None:
        klassnamn = KLASSER

    # --- Python-callback som JS anropar ---
    def klassificera_callback(img_b64):
        binary    = base64.b64decode(img_b64.split(',')[1])
        img_array = np.frombuffer(binary, dtype=np.uint8)
        img_bgr   = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        img_rgb   = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_proc  = forbehandla_bild(img_rgb)
        img_input = np.expand_dims(img_proc, axis=0)
        pred      = model.predict(img_input, verbose=0)[0]
        return {k: float(v) for k, v in zip(klassnamn, pred)}

    colab_output.register_callback('klassificera_bild', klassificera_callback)

    klass_json = json.dumps(klassnamn)
    emoji_map  = json.dumps({'rock': '✊', 'paper': '✋', 'scissors': '✌️'})

    html = f"""
<div id="rps-wrap" style="font-family: 'Courier New', monospace; max-width: 400px;">
  <video id="rps-video" width="320" height="240" autoplay playsinline
         style="border: 2px solid #555; border-radius: 6px; display:block;"></video>
  <div id="rps-pred" style="margin-top:10px; font-size:15px; line-height:2.2;">
    Startar kamera...
  </div>
</div>

<script>
(async () => {{
  const video   = document.getElementById('rps-video');
  const predDiv = document.getElementById('rps-pred');
  const klasser = {klass_json};
  const emojis  = {emoji_map};

  const stream  = await navigator.mediaDevices.getUserMedia({{video: true}});
  video.srcObject = stream;
  await video.play();

  const canvas  = document.createElement('canvas');
  canvas.width  = video.videoWidth  || 320;
  canvas.height = video.videoHeight || 240;

  async function uppdatera() {{
    canvas.getContext('2d').drawImage(video, 0, 0, canvas.width, canvas.height);
    const imgData = canvas.toDataURL('image/jpeg', 0.7);

    let result;
    try {{
      result = await google.colab.kernel.invokeFunction(
        'klassificera_bild', [imgData], {{}}
      );
    }} catch (e) {{
      predDiv.innerHTML = '<span style="color:red;">Fel: ' + e.message + '</span>';
      return;
    }}

    const preds = result.data['application/json'];

    let bästaKlass = klasser[0];
    let bästaVärde = 0;
    for (const k of klasser) {{
      if (preds[k] > bästaVärde) {{ bästaVärde = preds[k]; bästaKlass = k; }}
    }}

    let html = '';
    for (const k of klasser) {{
      const pct    = Math.round(preds[k] * 100);
      const filled = Math.round(preds[k] * 20);
      const bar    = '█'.repeat(Math.max(0, filled)) +
                     '░'.repeat(Math.max(0, 20 - filled));
      const isTop  = (k === bästaKlass);
      const color  = isTop
        ? (pct >= 60 ? '#22c55e' : '#f59e0b')
        : '#9ca3af';
      const weight = isTop ? 'bold' : 'normal';
      const emoji  = emojis[k] || '?';
      html += `<div style="color:${{color}};font-weight:${{weight}};">`
            + `${{emoji}} ${{k.padEnd(9)}} ${{bar}} ${{String(pct).padStart(3)}}%</div>`;
    }}
    predDiv.innerHTML = html;
  }}

  setInterval(uppdatera, 500);
}})();
</script>
"""
    display(HTML(html))
    print("Kameran är igång! Håll upp handen framför kameran. ✓")

In [ ]:
starta_kamera(model)